<h1 style="color:#b57c01;">[Data Cleaning] 財政部出口統計</h1>

In [5]:
import pandas as pds
import os
import re
from dotenv import load_dotenv

csv_ouptut_file_name ="mof_export_statistic.csv"


load_dotenv()

data_path = os.environ.get("DOWNLOAD_DIR")
print ( f"data path: {data_path}")

csvfiles = os.listdir( data_path )
csvfiles_number = len ( csvfiles )
print ( f"files ( {csvfiles_number} ) : {csvfiles}" )




data path: /home/botsz/AIPE02/ESG_carbon_tax/tool_kit/mof_export_statistics/raw_data/
files ( 132 ) : ['mof_statistic_112_9.csv', 'mof_statistic_114_2.csv', 'mof_statistic_106_1.csv', 'mof_statistic_106_4.csv', 'mof_statistic_107_10.csv', 'mof_statistic_111_7.csv', 'mof_statistic_110_2.csv', 'mof_statistic_112_1.csv', 'mof_statistic_106_2.csv', 'mof_statistic_112_8.csv', 'mof_statistic_108_8.csv', 'mof_statistic_107_5.csv', 'mof_statistic_114_11.csv', 'mof_statistic_108_2.csv', 'mof_statistic_111_8.csv', 'mof_statistic_114_5.csv', 'mof_statistic_109_10.csv', 'mof_statistic_107_8.csv', 'mof_statistic_110_3.csv', 'mof_statistic_108_7.csv', 'mof_statistic_111_4.csv', 'mof_statistic_105_11.csv', 'mof_statistic_105_3.csv', 'mof_statistic_109_2.csv', 'mof_statistic_106_3.csv', 'mof_statistic_112_2.csv', 'mof_statistic_114_8.csv', 'mof_statistic_105_12.csv', 'mof_statistic_108_10.csv', 'mof_statistic_109_12.csv', 'mof_statistic_106_12.csv', 'mof_statistic_113_3.csv', 'mof_statistic_107_7.csv'

In [6]:
fPath = os.path.join ( data_path, csvfiles[0] ) 
df=pds.read_csv( fPath , encoding="big5" )
unique_values = df["國家/地區別"].unique()
unique_values

array(['<b>全球</b>', '亞洲', '中國大陸與香港', '中國大陸', '香港', '日本', '南韓',
       '新南向18國(註1)', '東協10國', '泰國', '馬來西亞', '印尼', '菲律賓', '新加坡', '越南',
       '汶萊', '緬甸', '柬埔寨', '寮國', '南亞', '印度', '巴基斯坦', '孟加拉', '斯里蘭卡', '尼泊爾',
       '不丹', '哈薩克', '中東', '伊朗', '伊拉克', '以色列', '科威特', '阿曼', '卡達', '沙烏地阿拉伯',
       '土耳其', '阿拉伯聯合大公國', '北美洲', '美國', '加拿大', '中美洲', '墨西哥', '南美洲', '巴西',
       '智利', '秘魯', '歐洲', '德國', '法國', '荷蘭', '英國', '俄羅斯', '奧地利', '比利時',
       '匈牙利', '義大利', '波蘭', '西班牙', '瑞典', '瑞士', '烏克蘭', '歐盟(註2)', '東歐', '非洲',
       '阿爾及利亞', '安哥拉', '埃及', '南非', '肯亞', '奈及利亞', '大洋洲', '澳大利亞', '巴布亞紐幾內亞',
       '紐西蘭', nan], dtype=object)

### Merge the CSV files ###

In [7]:
def parse_month( x ):
    
    if pds.isna ( x ):
        return pds.NA
    
    monthStr = str ( x )

    m_range = re.search ( r"\(?(\d+)\s*~\s*(\d+)月\)?", x )
    if m_range :
        return int ( m_range.group(1) ) + 100
    
    m_single = re.search(r"(\d+)月", x)
    if m_single:
        return int(m_single.group(1))
    
    return pds.NA


In [8]:

def data_cleaning ( dfs , csvfiles ):

    for fname in csvfiles :

        m = re.search(r"mof_statistic_(\d+)_(\d+)\.csv", fname)
        if not m:
            continue

        year  = int(m.group(1))
        month = int(m.group(2))

        fPath = os.path.join ( data_path, fname ) 
        tmp = pds.read_csv ( fPath , encoding="big5" )

        tmp[ "月" ] = tmp ["統計期"].apply ( parse_month ).astype( "Int64" )

        tmp = tmp [ tmp [ "月" ] < 13 ]

        locationStrList = [ "<b>全球</b>" , "中國大陸與香港" ]
        for l in locationStrList :
            tmp = tmp [ tmp [ "國家/地區別" ] != l ]
        
        tmp = tmp [~tmp [ "國家/地區別" ].isna()]
        

        # add year and month
        tmp[ "年" ] = year
        cols_old = list( tmp.columns )
        cols = [[ "年", "月" ] , cols_old ] 

        dfs.append( tmp )

dfs= []
data_cleaning ( dfs , csvfiles ) 
df = pds.concat ( dfs, ignore_index=True )
df = df.rename(columns={"<b>總計</b>": "總計"})


print(df.columns.tolist())
df_selected = df[["統計期","年","月","國家/地區別", "總計"]].copy()
df_selected



['統計期', '國家/地區別', '總計', '1.活動物；動物產品', '(1)水產品', '2.植物產品', '(1)食用蔬菜', '(2)食用果實及堅果', '3.動植物油脂', '4.調製食品；飲料及菸酒', '(1)調製食品', '(2)飲料、酒類及醋', '(3)菸及菸葉代用品', '5.礦產品', '(1)石油煉製品', '6.化學品', '(1)有機化學品', '7.塑膠、橡膠及其製品', '(1)塑膠原料', '(2)塑膠製品', '(3)橡膠及其製品', '8.毛皮及其製品', '9.木及木製品', '10.紙漿；紙及其製品；印刷品', '11.紡織品', '(1)紗布', '12.鞋、帽及其他飾品', '13.非金屬礦物製品', '14.珠寶及貴金屬製品', '15.基本金屬及其製品', '(1)鋼鐵及其製品', '(2)其他基本金屬及其製品', '16.機械及電機設備', '(1)電子零組件', '(2)機械', '(3)電機產品', '(4)資通與視聽產品', '(5)家用電器', '17.運輸工具', '18.光學及精密儀器；鐘錶；樂器', '(1)光學及精密儀器', '19.其他', '(1)家具', '(2)玩具與運動用品', '月', '年']


,統計期,年,月,國家/地區別,總計
0,112年 9月,112,9,亞洲,25612272.0
1,112年 9月,112,9,中國大陸,8134400.0
2,112年 9月,112,9,香港,5689049.0
3,112年 9月,112,9,日本,2191887.0
4,112年 9月,112,9,南韓,1432213.0
...,...,...,...,...,...
9499,112年 12月,112,12,奈及利亞,6014.0
9500,112年 12月,112,12,大洋洲,367496.0
9501,112年 12月,112,12,澳大利亞,272288.0
9502,112年 12月,112,12,巴布亞紐幾內亞,5566.0


In [9]:
processed_data_path = os.environ.get("PROCESSED_DATA_DIR") 
csv_ouptut_file_name = os.path.join ( processed_data_path , csv_ouptut_file_name )
df.to_csv ( csv_ouptut_file_name, index=False, encoding="utf-8" )

df = pds.read_csv ( csv_ouptut_file_name )
df.head()

,統計期,國家/地區別,總計,1.活動物；動物產品,(1)水產品,2.植物產品,(1)食用蔬菜,(2)食用果實及堅果,3.動植物油脂,4.調製食品；飲料及菸酒,...,(4)資通與視聽產品,(5)家用電器,17.運輸工具,18.光學及精密儀器；鐘錶；樂器,(1)光學及精密儀器,19.其他,(1)家具,(2)玩具與運動用品,月,年
0,112年 9月,亞洲,25612272.0,108735,98451,31053,5738,2591,7045,133327,...,2838529,12308,158577,652318,643735,189570,27415,55425,9,112
1,112年 9月,中國大陸,8134400.0,16518,12259,3093,47,60,257,35458,...,1023007,3466,49443,273964,271683,44997,2500,16758,9,112
2,112年 9月,香港,5689049.0,10206,9918,4985,394,1692,1218,28473,...,480524,900,3927,53382,50301,32845,1092,8973,9,112
3,112年 9月,日本,2191887.0,40536,36940,13142,5175,260,819,10960,...,251533,3908,47733,89108,87233,48080,19284,14839,9,112
4,112年 9月,南韓,1432213.0,2270,2103,1005,1,118,2739,7417,...,72297,803,9227,85280,84894,9470,1077,4027,9,112
